In [12]:
import getpass
from datetime import datetime
from pathlib import Path

import requests
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from ipyleaflet import GeoJSON, Map, basemaps
from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    MimeType,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    SHConfig,
)
import xarray as xr
import rioxarray  # This registers the 'rasterio' engine
# from sklearn.metrics import accuracy_score
config = SHConfig()
config.sh_client_id = "sh-0470ee77-ae47-4290-a69b-1dd95df154b8"
config.sh_client_secret = "IoKB7Ia7iIQM7AnoYm4H4H8aIWNwST4J"
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"

In [24]:
evalscript_cloudless = """
//VERSION=3
function setup() {
    return {
        input: ["B08", "B04", "B03", "B02", "B8A", "B11", "SCL"],
        output: {
            bands: 5,
            sampleType: "INT16"
        },
        mosaicking: "ORBIT"
    }
}

function getFirstQuartileValue(values) {
    values.sort((a,b) => a-b);
    return getFirstQuartile(values);
}

function getFirstQuartile(sortedValues) {
    var index = Math.floor(sortedValues.length / 4);
    return sortedValues[index];
}

function validate(sample) {
    // Define codes as invalid:
    const invalid = [
        0, // NO_DATA
        1, // SATURATED_DEFECTIVE
        3, // CLOUD_SHADOW
        7, // CLOUD_LOW_PROBA
        8, // CLOUD_MEDIUM_PROBA
        9, // CLOUD_HIGH_PROBA
        10 // THIN_CIRRUS
    ]
    return !invalid.includes(sample.SCL)
}

function evaluatePixel(samples) {
    var valid = samples.filter(validate);
    if (valid.length > 0 ) {
        let cloudless = {
            b08: getFirstQuartileValue(valid.map(s => s.B08)),
            b11: getFirstQuartileValue(valid.map(s => s.B11)),
            b04: getFirstQuartileValue(valid.map(s => s.B04)),
            b03: getFirstQuartileValue(valid.map(s => s.B03)),
            b02: getFirstQuartileValue(valid.map(s => s.B02)),
            b8A: getFirstQuartileValue(valid.map(s => s.B8A)),
        }
        let ndvi = ((cloudless.b08 - cloudless.b04) / (cloudless.b08 + cloudless.b04))
        let moisture_idx = ((cloudless.b8a - cloudless.b11) / (cloudless.b8a+ cloudless.b11))
        // This applies a scale factor so the data can be saved as an int
        let scale = [cloudless.b04, cloudless.b03, cloudless.b02, ndvi, moisture_idx].map(v => v*10000);
        return scale
    }
    // If there isn't enough data, return NODATA
    return [-32768, -32768, -32768, -32768]
}
"""
def interval_of_interest(year):
    return (datetime(year, 6, 1), datetime(year, 9, 1))


def get_request(year, coors):
    time_interval = interval_of_interest(year)
    epsg = 3035
    bbox = BBox(coors, CRS(4326)).transform(epsg)
    return SentinelHubRequest(
        evalscript=evalscript_cloudless,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L2A.define_from(
                    "s2", service_url=config.sh_base_url
                ),
                time_interval=time_interval,
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=bbox,
        resolution=(10, 10),
        config=config,
        data_folder="./data",
    )


In [25]:
sh_requests = {}
bbox_coords = [14.880833, 54.044444, 14.95, 54.068889]
for year in range(2023, 2024):
    sh_requests[year] = get_request(year, bbox_coords)
list_of_requests = [request.download_list[0] for request in sh_requests.values()]
data = SentinelHubDownloadClient(config=config).download(
    list_of_requests, max_threads=5
)
def request_output_path(request):
    return Path(request.data_folder, request.get_filename_list()[0])

for year, request in sh_requests.items():
    request_output_path(request).rename(f"./data/{year}.tif")

In [26]:
def add_time_dim(xda):
    # This pre-processes the file to add the correct
    # year from the filename as the time dimension
    year = int(Path(xda.encoding["source"]).stem)
    return xda.expand_dims(year=[year])
tiff_paths = Path("./data").glob("*.tif")
ds_s2 = xr.open_mfdataset(
    tiff_paths,
    engine="rasterio",
    preprocess=add_time_dim,
    band_as_variable=True,
)
if not ds_s2.rio.crs:
    ds_s2 = ds_s2.rio.write_crs("EPSG:32633")
ds_s2 = ds_s2.rio.reproject("EPSG:4326")
ds_s2 = ds_s2.rename({'x': 'longitude', 'y': 'latitude'})
# ds_s2.to_dataframe().reset_index().to_csv("mapped_data_leniwa_pizdo.csv")

Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as 

In [16]:
# if not ds_s2.rio.crs:
#     # Assign the CRS (replace 'EPSG:32633' with your actual CRS)
#     ds_s2 = ds_s2.rio.write_crs("EPSG:32633")
# ds_s2 = ds_s2.rio.reproject("EPSG:4326")
# ds_s2 = ds_s2.rename({'x': 'longitude', 'y': 'latitude'})
# ds_s2.to_dataframe().reset_index().to_csv("mapped_data_leniwa_pizdo.csv")

In [29]:
ds_s2

<xarray.Dataset> Size: 21MB
Dimensions:      (longitude: 525, latitude: 226, year: 9)
Coordinates:
  * longitude    (longitude) float64 4kB 14.88 14.88 14.88 ... 14.95 14.95 14.95
  * latitude     (latitude) float64 2kB 54.07 54.07 54.07 ... 54.04 54.04 54.04
  * year         (year) int64 72B 2015 2016 2017 2018 2019 2020 2021 2022 2023
    spatial_ref  int64 8B 0
Data variables:
    band_1       (year, latitude, longitude) float32 4MB nan nan nan ... nan nan
    band_2       (year, latitude, longitude) float32 4MB nan nan nan ... nan nan
    band_3       (year, latitude, longitude) float32 4MB nan nan nan ... nan nan
    band_4       (year, latitude, longitude) float32 4MB nan nan nan ... nan nan
    band_5       (year, latitude, longitude) float32 4MB nan nan nan ... nan nan
Attributes:
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    TIFFTAG_RESOLUTIONUNIT:  1 (unitless)
    AREA_OR_POINT:           Area

In [30]:
df=ds_s2.band_5.to_dataframe().reset_index()
df

,year,latitude,longitude,spatial_ref,band_5
0,2015,54.071465,14.880899,0,NaN
1,2015,54.071465,14.881030,0,NaN
2,2015,54.071465,14.881162,0,NaN
3,2015,54.071465,14.881294,0,NaN
4,2015,54.071465,14.881425,0,NaN
...,...,...,...,...,...
1067845,2023,54.041850,14.949342,0,NaN
1067846,2023,54.041850,14.949474,0,NaN
1067847,2023,54.041850,14.949606,0,NaN
1067848,2023,54.041850,14.949737,0,NaN


In [27]:
df=ds_s2.band_4.to_dataframe().reset_index()
df

,year,latitude,longitude,spatial_ref,band_4
0,2015,54.071465,14.880899,0,NaN
1,2015,54.071465,14.881030,0,NaN
2,2015,54.071465,14.881162,0,NaN
3,2015,54.071465,14.881294,0,NaN
4,2015,54.071465,14.881425,0,NaN
...,...,...,...,...,...
1067845,2023,54.041850,14.949342,0,NaN
1067846,2023,54.041850,14.949474,0,NaN
1067847,2023,54.041850,14.949606,0,NaN
1067848,2023,54.041850,14.949737,0,NaN


In [18]:
(df['y']/df['y'].max()).min()

KeyError: 'y'